### Name Entity From Capital Letter Words

In [ ]:
#!pip install kneed

In [66]:
import pandas as pd
import numpy as np
import re
from kneed import KneeLocator

#### Capital Letter Word And Lower Letter Word

In [67]:
df = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/French/Result/Word/Data/Not_Apply_Lower_Word1.csv") # Not apply lower technique
df

,word,frequency
0,de,3981257
1,pas,2693790
2,Je,2606215
3,le,2262594
4,la,2187456
...,...,...
597028,commelundesplusieursvictimes,1
597029,Morceauxassezpourfairequatre,1
597030,WilliamEastonaété,1
597031,quivousdétruire,1


In [68]:
df = df[df.iloc[:,1]>=5]

In [69]:
def word_length(var):
    var = str(var)
    if len(var) <= 12:
        return var
    else:
        return "word is bigger than adjust value"

In [70]:
df = pd.DataFrame(df.iloc[:,0].apply(lambda x: word_length(x)))

In [71]:
df.drop(df[df.iloc[:,0] == "word is bigger than adjust value"].index, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,word
0,de
1,pas
2,Je
3,le
4,la
...,...
221091,spécieuse
221092,Yippitykiyay
221093,galbes
221094,Gaelou


In [72]:
capital_word_list = []
lower_word_list = []
for i in df.iloc[:,0]:
    i = str(i)
    if i[0].isupper():
        capital_word_list.append(i)
    else:
        lower_word_list.append(i)

In [73]:
df_capital = pd.DataFrame(capital_word_list)
df_capital.rename(columns={0:"word"}, inplace=True)
df_capital = pd.DataFrame(df_capital.iloc[:,0].apply(lambda x: x.capitalize()))
df_capital

,word
0,Je
1,Cest
2,Tu
3,Il
4,Vous
...,...
102094,Wyvold
102095,Rodéo
102096,Amateur
102097,Yippitykiyay


In [74]:
df_capital_to_lower = pd.DataFrame(df_capital.iloc[:,0].apply(lambda x : x.lower()))
df_capital_to_lower

,word
0,je
1,cest
2,tu
3,il
4,vous
...,...
102094,wyvold
102095,rodéo
102096,amateur
102097,yippitykiyay


In [75]:
df_lower = pd.DataFrame(lower_word_list)
df_lower.rename(columns={0:"word"}, inplace=True)
df_lower

,word
0,de
1,pas
2,le
3,la
4,que
...,...
118992,jaifait
118993,impatienter
118994,spécieuse
118995,galbes


In [76]:
df_lower_to_lower = pd.DataFrame(df_lower.iloc[:,0].apply(lambda x : x.lower()))
df_lower_to_lower

,word
0,de
1,pas
2,le
3,la
4,que
...,...
118992,jaifait
118993,impatienter
118994,spécieuse
118995,galbes


In [ ]:
capital_set = set(df_capital_to_lower.iloc[:,0])

In [ ]:
lower_set = set(df_lower_to_lower.iloc[:,0])

In [ ]:
df_all_capital = pd.DataFrame(capital_set.difference(lower_set)) # compare capital letter word and lower letter word
df_all_capital.rename(columns={0:"word"}, inplace=True)
#df_all_capital = pd.DataFrame(df_all_capital.iloc[:,0].apply(lambda x: x.capitalize()))
df_all_capital

In [ ]:
# Check result with all capital word
# df_merge = pd.merge(df_capital, df_all_capital, how="inner", on="word")
# df_merge.drop_duplicates(inplace=True)
# df_merge.reset_index(drop=True, inplace=True)
# df_merge

In [ ]:
df_all_capital.to_excel("All_Capital_Letter_Word.xlsx", sheet_name="Word", index=False)

#### Sentence First Word Analysis

In [ ]:
df_sent = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/French/Result/Sentence/Merge/Sentence_Merge_Result2.csv")
df_sent

In [ ]:
#df_sent = df_sent[df_sent.iloc[:,1]>=5]

In [ ]:
first_words = []
for i in df_sent.iloc[:,0]: # Select sentence first word
    words = re.findall("\w+", i)
    first_words.append(words[0])

In [ ]:
first_ser = pd.Series(first_words)
first_ser

In [ ]:
df_first = pd.DataFrame(first_ser.value_counts(ascending=False))
df_first.reset_index(inplace=True)
df_first.rename(columns={"index":"word", 0:"frequency"}, inplace=True)
df_first

In [ ]:
set_first = set(df_first.iloc[:,0])

In [ ]:
set_all_capital = set(df_all_capital.iloc[:,0])

In [ ]:
df_all_spec = pd.DataFrame(set_all_capital.difference(set_first)) # Capital letter word and first sentence word
df_all_spec.rename(columns={0:"word"}, inplace=True)
df_all_spec

In [ ]:
df_all_spec.to_excel("Special_Name_Exactly.xlsx", sheet_name="Spec_Name", index=False)

#### Spec Name Knee Point

In [ ]:
df_word = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/French/Result/Word/Merge/French_Word_Merge_Result2.csv")
df_word

In [ ]:
df_all_spec_freq = pd.merge(df_word, df_all_spec, how="inner", on="word")
df_all_spec_freq

In [ ]:
total_freq_spec = df_all_spec_freq.iloc[:,1].sum()
total_freq_spec

In [ ]:
df_all_spec_freq["ratio"] = round(((df_all_spec_freq.iloc[:,1]/total_freq_spec)*100),7)
df_all_spec_freq

In [ ]:
df_all_spec_freq["cumul_ratio"] = np.cumsum(df_all_spec_freq.iloc[:,2])
df_all_spec_freq

In [ ]:
spec_kneedle = KneeLocator(x=df_all_spec_freq.cumul_ratio.index, y=df_all_spec_freq.cumul_ratio, S=1.0, curve="concave", direction="increasing")
spec_kneedle.plot_knee()

In [ ]:
print(round(spec_kneedle.knee_y))

In [ ]:
knee_point_spec = round(spec_kneedle.knee_y)

In [ ]:
df_spec_knee = df_all_spec_freq[df_all_spec_freq.cumul_ratio <= knee_point_spec]
df_spec_knee

In [ ]:
df_spec_knee.to_excel("Special_Name_Exactly_Knee.xlsx", sheet_name="Spec_Name", index=False)

#### Sentence First Word Knee Point

In [ ]:
df_first = df_first[df_first.iloc[:,1]>=5]

In [ ]:
total_freq_first = df_first.iloc[:,1].sum()
total_freq_first

In [ ]:
df_first["ratio"] = round(((df_first.iloc[:,1]/total_freq_first)*100),7)
df_first

In [ ]:
df_first["cumul_ratio"] = np.cumsum(df_first.iloc[:,2])
df_first

In [ ]:
first_kneedle = KneeLocator(x=df_first.cumul_ratio.index, y=df_first.cumul_ratio, S=1.0, curve="concave", direction="increasing")
first_kneedle.plot_knee()

In [ ]:
print(round(first_kneedle.knee_y))

In [ ]:
knee_point_first = round(first_kneedle.knee_y)

In [ ]:
df_first_knee = df_first[df_first.cumul_ratio <= knee_point_first]
df_first_knee

In [ ]:
#df_first_knee.to_excel("First_Word_Knee.xlsx", sheet_name="Word", index=False)

In [ ]:
df_first_no_knee = df_first[df_first.cumul_ratio > knee_point_first]
df_first_no_knee.reset_index(drop=True, inplace=True)
df_first_no_knee = df_first_no_knee.iloc[:,0:2]
df_first_no_knee

In [ ]:
#df_first_no_knee.to_excel("First_Word_No_Knee.xlsx", sheet_name="Word", index=False)

In [ ]:
set_first_knee = set(df_first_knee.iloc[:,0])

In [ ]:
set_first_no_knee = set(df_first_no_knee.iloc[:,0])

In [ ]:
#set_all_capital = set(df_all_capital.iloc[:,0])

In [ ]:
df_all_knee = pd.DataFrame(set_all_capital.difference(set_first_knee)) # Capital letter word and first sentenc word knee analysis
df_all_knee.rename(columns={0:"word"}, inplace=True)
df_all_knee

#### First 40 000 Word And Capital Letter Word Analysis

In [ ]:
df_word = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/French/Result/Word/Merge/French_Word_Merge_Result2.csv")
df_word = df_word.head(40000)
df_word

In [ ]:
set_word = set(df_word.iloc[:,0])

In [ ]:
set_all_knee = set(df_all_knee.iloc[:,0])

In [ ]:
df_word_capital = pd.DataFrame(set_word.intersection(set_all_knee))
df_word_capital

In [ ]:
df_word_capital.to_excel("Word_And_Capital_Letter_Word_Intersect.xlsx", sheet_name="Word_Cağital", index=False)